In [1]:
# -*- coding: utf-8 -*-
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import bs4
import pandas as pd
import numpy as np
import time
import os
from tqdm import tqdm_notebook
from IPython.display import clear_output
import chromedriver_autoinstaller
from selenium import webdriver
from tqdm import tqdm

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''
< naver 뉴스 전문 가져오기 >_select 사용
- 네이버 뉴스만 가져와서 결과값 조금 작음 
- 결과 메모장 저장 -> 엑셀로 저장 

'''''''''''''''''''''''''''''''''''''''''''''''''''''''''

def get_news(n_url):
    
    news_detail = []
    time.sleep(0.1)
    headers = {"User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36'}
    resp = requests.get(n_url, headers=headers)
    soup = BeautifulSoup(resp.text, 'html.parser')

    try:
        
        title = soup.select('#articleTitle')[0].text
        news_detail.append(title)
#         print(title)
    
        report = soup.select('#articleBodyContents')[0].text
        news_detail.append(report)
#         print(report)
    
        company1 = soup.select('#main_content > div.article_header > div.press_logo > a > img')[0]
        company = company1.get('title')
        news_detail.append(company)
#         print(company)

        date = soup.select('#main_content > div.article_header > div.article_info > div > span.t11')[0].text.split(' ')[0]
        news_detail.append(date)
#         print(date)
    
    except:
        return None
    
    return news_detail

In [6]:
def crawler(maxpage,query,s_date,e_date):

    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")
    page = 1
    maxpage_t =(int(maxpage)-1)*10+1   # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지

    all_urls = []
    real_all_urls = []
    
    
    while page < maxpage_t:
    
        #print(page)
    

        url = "https://search.naver.com/search.naver?where=news&query="+ query +"&sort=2&photo=3&field=0&pd=3&ds="+ s_date +'&de='+ e_date +'&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from'+ s_from +'to'+ e_to +',a:all&start='+ str(page)
        
        req = requests.get(url)
        #print(url)
        cont = req.content
        soup = BeautifulSoup(cont, 'html.parser')
        #print(soup)
        
        
        for urls in soup.select(".info"):
            try :
#                 print(urls["href"])
                if urls["href"].startswith("https://news.naver.com"):
                    all_urls.append(urls["href"])
#                     print(urls["href"])

                    
#                     print(all_urls)
                    
                    
            except Exception as e:
#                 print(e)
                continue
                
        page += 10
    
    real_all_urls.extend(all_urls)
#     print(real_all_urls)

    add_news_dict = []
    for i in real_all_urls:
        
        news_detail = get_news(i)
        
        try:
        
    
            news_dict = {'title' : news_detail[0],
                         'report' : news_detail[1],
                         'company' : news_detail[2],
                         'date' : news_detail[3]}
#             print(news_dict)

            add_news_dict.append(news_dict)
        except:
            pass
#     print(add_news_dict)
        
        
        
    news_df = pd.DataFrame(add_news_dict)
    folder_path = os.getcwd()
    xlsx_file_name = '네이버뉴스_{}_{}_{}.xlsx'.format(query,s_date,e_date)

    news_df.to_excel(xlsx_file_name)
                    


def main():
    maxpage = input("최대 출력할 페이지수 입력하시오: ") 
    query = input("검색어 입력: ")
    s_date = input("시작날짜 입력(2019.01.01):")  #2019.01.01
    e_date = input("끝날짜 입력(2019.04.28):")   #2019.04.28
    crawler(maxpage,query,s_date,e_date) #검색된 네이버뉴스의 기사내용을 크롤링합니다. 
    
main()

최대 출력할 페이지수 입력하시오: 400
검색어 입력: CJ제일제당
시작날짜 입력(2019.01.01):2016.01.01
끝날짜 입력(2019.04.28):2016.12.31
